### 인터파크 - 베스트셀러

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

In [10]:
url = "http://book.interpark.com/display/collectlist.do?_method=BestsellerHourNew201605&bestTp=1&dispNo=028"
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}

- html 가져오기

In [17]:
result = requests.get(url, headers=header)
soup = BeautifulSoup(result.text, 'html.parser')

- 리스트 가져오기

In [18]:
lis = soup.select('.rankBestContentList > ol > li')
len(lis)

15

In [19]:
li = lis[0]
li

<li>
<!-- listItem | singleType or setType :: start -->
<div class="listItem singleType">
<div class="cover">
<div class="coverImage">
<label>
<a href="/product/BookDisplay.do?_method=detail&amp;sc.shopNo=0000400000&amp;sc.prdNo=356522531&amp;sc.saNo=003003001&amp;bid1=Best_zone&amp;bid2=LiveRanking&amp;bid3=PRD&amp;bid4=001" target="_blank">
<span class="checkBox">
<input name="_KEY" type="checkbox" value="356522531"/>
</span>
<img alt="세이노의 가르침" src="http://bimage.interpark.com/goods_image/2/5/3/1/356522531s.jpg"/>
<i class="decoFront"></i>
</a>
</label>
</div>
<div class="coverOption">
<ul>
<li class="cart">
<a class="cartGo" href="#" value="356522531"><span>CART</span></a>
</li>
<li class="buy">
<a href="javascript:directOrd('356522531');"><span>BUY</span></a>
</li>
</ul>
</div>
<div class="rankNumber digit2">
<span class="rankBtn_ctrl rkNum_B01"></span>
</div>
</div>
<div class="itemIcon">
<span class="bestSeller">베스트셀러</span>
</div>
<a href="/product/BookDisplay.do?_method=detail

In [30]:
# 랭크 가져오기
# 랭크값이 1개
li = lis[0]
ranks = li.select_one('.rankNumber > span')['class'][1][-1]
print(ranks)
# for rank in ranks:
#     print(rank)

1


In [41]:
# 랭크 값이2
li = lis[12]
ranks = li.select('.rankNumber > span')
for rank in ranks:
    print(rank['class'][1][-1], end='')

13

In [45]:
# 랭크 값이2
li = lis[12]
ranks = li.select('.rankBtn_ctrl')
rank_data = ''
for rank in ranks:
    rank_data += rank['class'][-1][-1]
    
print(int(rank_data))

13


In [46]:
# 랭크 함수
def get_rank(ranks):
    
    if len(ranks) == 1:
        return int(rank['class'][-1][-1])
    else:
        return int(rank['class'][0][-1] + rank['class'][-1][-1])


In [47]:
# 제목
title = li.select_one('.itemName').get_text().strip()
title

'사랑해 사랑해 사랑해(출간 15주년 기념판)'

In [49]:
# 저자
author = li.select_one('.author').get_text().strip()
author

'버나뎃 로제티 슈스탁/신형건 역/캐롤라인 제인 처치 그림'

In [50]:
# 출판사
company = li.select_one('.company').get_text().strip()
company

'보물창고'

In [53]:
# 가격 
price = int(li.select_one('.price > em').get_text().strip().replace(',', ''))
price

10620

- 각각의 정보들 가져오기

In [60]:
lines = []
for li in tqdm(lis):
    
    rank_data = li.select('.rankBtn_ctrl')
    if len(rank_data) == 1:
        rank = int(rank_data[0]['class'][-1][-1])
    else:
        rank = int(rank_data[0]['class'][-1][-1] + rank_data[-1]['class'][-1][-1])

    title = li.select_one('.itemName').get_text().strip()
    author = li.select_one('.author').get_text().strip()
    company = li.select_one('.company').get_text().strip()
    price = li.select_one('.price > em').get_text().strip()
    lines.append({'제목':title, '저자': author, '랭크': rank, '출판사': company, '판매가': price})

df = pd.DataFrame(lines)      
df


100%|██████████| 15/15 [00:00<00:00, 441.43it/s]


,제목,저자,랭크,출판사,판매가
0,세이노의 가르침,세이노,1,데이원,"6,480"
1,이지성의 1만 킬로미터,이지성,2,차이정원,"16,200"
2,건강과 다이어트를 동시에 잡는 7대 3의 법칙 채소·과일식,조승우,3,바이북스,"17,100"
3,흔한남매 13,"흔한남매, 백난도/유난희 그림/흔한컴퍼니 감수",4,미래엔아이세움,"13,050"
4,고래,천명관,4,문학동네,"13,500"
5,오십의 기술,이호선,6,카시오페아,"16,200"
6,감정에 휘둘리는 아이 감정을 잘 다루는 아이,손승현,7,빅피시,"15,120"
7,몰입 합본판,,7,알에이치코리아,"28,800"
8,요즘 어른을 위한 최소한의 맞춤법,이주윤,7,빅피시,"15,120"
9,설민석의 한국사 대모험 24,"설민석, 스토리박스/정현희 그림/강석화 감수",7,단꿈아이,"11,700"


In [ ]:
# 현재날짜로 저장
now = datetime.now()
cur_day = now.strftime('%Y%m%d')

df.to_csv(f'data/{cur_day}_인터파크베스트.csv', index=False)